In [3]:
import tensorflow as tf
print(tf.__version__)
import cv2
print(cv2.__version__)
import time
import os
import numpy as np
import posenet

c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.4.0
3.4.5


# Function for calculating angle between two vectors

In [4]:

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return 180*(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))/np.pi

# Parameters

In [5]:
path_input = './exercises'
path_output = './output'
model_number = 1
scale_factor = 0.1
exercise = 'pushup.avi'
body_points = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}
body_points_score = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}

# Image Analysis

In [22]:
filenames = [f.path for f in os.scandir(path_input) if f.is_file() and f.path.endswith(('.png', '.jpg'))]
print(filenames)
start = time.time()
with tf.Session() as sess:
    model_cfg, model_outputs = posenet.load_model(model_number, sess)
    output_stride = model_cfg['output_stride']
    for f in filenames:
        print(f)
        input_image, draw_image, output_scale = posenet.read_imgfile(
            f, scale_factor=scale_factor, output_stride=output_stride)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale


        draw_image = posenet.draw_skel_and_kp(
            draw_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.25, min_part_score=0.25)

        cv2.imwrite(os.path.join(path_output, os.path.relpath(f, path_input)), draw_image)

        
        print("Results for image: %s" % f)
        for pi in range(len(pose_scores)):
            if pose_scores[pi] == 0.:
                break
            print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
            for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                if posenet.PART_NAMES[ki] in body_points:
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
        #Calculate angle between shoulder and wrist using the elbow as the origin
        angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
        angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
        
        #Calculate the average confidence score of the shoulder, elbow, and wrist
        confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
        confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
        confidence_right /= 3
        confidence_left /= 3
        
        print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
        print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
        print()
    print('Average FPS:', len(filenames) / (time.time() - start))


['./exercises\\pullup (1).jpg', './exercises\\pullup (10).jpg', './exercises\\pullup (11).jpg', './exercises\\pullup (12).jpg', './exercises\\pullup (13).jpg', './exercises\\pullup (14).jpg', './exercises\\pullup (2).jpg', './exercises\\pullup (3).jpg', './exercises\\pullup (4).jpg', './exercises\\pullup (5).jpg', './exercises\\pullup (6).jpg', './exercises\\pullup (7).jpg', './exercises\\pullup (8).jpg', './exercises\\pullup (9).jpg', './exercises\\pushup (1).jpg', './exercises\\pushup (2).jpg', './exercises\\pushup (3).jpg', './exercises\\pushup (4).jpg', './exercises\\pushup (5).jpg', './exercises\\pushup (6).jpg', './exercises\\pushup (7).jpg', './exercises\\pushup (8).jpg', './exercises\\pushup (9).jpg', './exercises\\standing (1).jpg', './exercises\\standing (2).jpg', './exercises\\standing (3).jpg', './exercises\\standing (4).jpg', './exercises\\standing (5).jpg']


C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\posenet\converter\config.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(cfg_f)


./exercises\pullup (1).jpg
Results for image: ./exercises\pullup (1).jpg
Pose #0, score = 0.434485
Angle Right: 63.299983 degrees with confidence 0.515527
Angle Left: 61.855652 degrees with confidence 0.554458

./exercises\pullup (10).jpg
Results for image: ./exercises\pullup (10).jpg
Pose #0, score = 0.526694
Angle Right: 36.661478 degrees with confidence 0.699724
Angle Left: 44.486245 degrees with confidence 0.562837

./exercises\pullup (11).jpg
Results for image: ./exercises\pullup (11).jpg
Pose #0, score = 0.420122
Angle Right: 118.984773 degrees with confidence 0.319526
Angle Left: 95.356602 degrees with confidence 0.282645

./exercises\pullup (12).jpg
Results for image: ./exercises\pullup (12).jpg
Pose #0, score = 0.385693
Angle Right: 85.713264 degrees with confidence 0.396059
Angle Left: 102.210275 degrees with confidence 0.376872

./exercises\pullup (13).jpg
Results for image: ./exercises\pullup (13).jpg
Pose #0, score = 0.522559
Angle Right: 43.281250 degrees with confidence 

# Video Analysis

In [1]:
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise)
        #cap.set(3, args.cam_width)
        #cap.set(4, args.cam_height)
        cap.set(3, 1080)
        cap.set(4, 1920)

        start = time.time()
        frame_count = 0
        min = 10000
        prev = 10000
        state = 0
        while True:
            input_image, display_image, output_scale = posenet.read_cap(
                cap, scale_factor=0.2, output_stride=output_stride)
            #output_stride
            #args.scale_factor
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)

            overlay_image = cv2.resize(overlay_image, (600, 900))

            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "leftEye":
                        print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        if c[0] > prev and state == 0:
                            prev = c[0]
                        elif c[0] < prev and state == 0:
                            state = 1
                        elif c[0] < min and state == 1:
                            min = c[0]
                        elif c[0] > min and state == 1:
                            print('FOUND MAX')
                            prev = c[0]
                            min = 10000
                            state = 0
                    if posenet.PART_NAMES[ki] in body_points:
                        body_points[posenet.PART_NAMES[ki]] = c
                        body_points_score[posenet.PART_NAMES[ki]] = s
            #Calculate angle between shoulder and wrist using the elbow as the origin
            angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
            angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            
            print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
            print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
            print()
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        print('Average FPS: ', frame_count / (time.time() - start))

NameError: name 'tf' is not defined